# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [3]:
#Your code here
import numpy as np
import pandas as pd

from scipy import stats
import seaborn as sns

df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [9]:
df.value_counts(df['action'])

action
view     6328
click    1860
dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [8]:
df.describe()

,id
count,8188.000000
mean,564699.749878
std,219085.845672
min,182988.000000
25%,373637.500000
50%,566840.500000
75%,758078.000000
max,937217.000000


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [18]:
#Your code here
control = df[df['group'] == 'control']
experiment = df[df['group'] == 'experiment']

control_view = control[control['action'] == 'view']
control_click = control[control['action'] == 'click']
experiment_view = experiment[experiment['action'] == 'view']
experiment_click = experiment[experiment['action'] == 'click']

control_ratio = len(control_click) / len(control_view)
print(control_ratio)

experiment_ratio = len(experiment_click) / len(experiment_view)
print(experiment_ratio)

0.2797118847539016
0.3097463284379172


In [20]:
counts = df.groupby(['group', 'action']).count()
counts

timestamp    id
group      action                 
control    click         932   932
           view         3332  3332
experiment click         928   928
           view         2996  2996

In [ ]:
df_A = df.loc[df['group'] == 'control']
df_B = df.loc[df['group'] == 'experiment']

In [21]:
control_clicksum = len(control_click)
experiment_clicksum = len(experiment_click)
control_clicksum, experiment_clicksum

(932, 928)

In [23]:
control_viewsum = len(control_view)
experiment_viewsum = len(experiment_view)
control_viewsum, experiment_view_sum

(3332, 2996)

In [24]:
contingency_table = np.array([[control_clicksum, control_viewsum],
                              [experiment_clicksum, experiment_viewsum]])

contingency_table

array([[ 932, 3332],
       [ 928, 2996]])

In [25]:
stats.chi2_contingency(contingency_table)

(3.636160051233291,
 0.056537191086915774,
 1,
 array([[ 968.61748901, 3295.38251099],
        [ 891.38251099, 3032.61748901]]))

In [41]:
#goodness of fit
experiment_chi = [928, 2996]
control_chi = [932, 3332]

result = stats.chisquare(f_obs=experiment_chi, f_exp=control_chi)
result

Power_divergenceResult(statistic=33.89952032315072, pvalue=5.8033006040559e-09)

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

In [ ]:
control_CR, experiment_CR = contingency_table[:, 0] / contingency_table.sum(axis=1)
abs_diff = (100*(experiment_CR-control_CR))
print(100*control_CR)
print(100*experiment_CR)
print(abs_diff)

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [32]:
#Your code here
expected_experiment_clicks = control_ratio * experiment_viewsum
expected_experiment_clicks

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [33]:
#Your code here
n = experiment_viewsum
p = control_ratio
var = n * p * (1-p)
std = np.sqrt(var)
print(std)

24.568547907005815


In [35]:
actual_experiment_clicks = experiment_clicksum
z_score = (actual_experiment_clicks - expected_experiment_clicks)/std
print(z_score)

3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [45]:
#Your code here
p_val = stats.norm.sf(z_score)
print(p_val)

0.00012486528006951198


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**
Yes, roughly, they are both significant, but our first p-value is a lot lower than the second. 
First p = 5.8033006040559e-09
Second p = 0.00012486528006951198

*Both p values would lead us to reject the Null Hypothesis

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.